In [76]:
import requests
import csv
import json
import time

url = 'http://api.olhovivo.sptrans.com.br/v0'
s = requests.session()

In [80]:
s.post(url+'/Login/Autenticar?token=<token_here>')

<Response [200]>

In [60]:
def get_cod_linha( linha ):
    res = s.get(url+'/Linha/Buscar?termosBusca='+linha)
    return res.json()[0]['CodigoLinha']

In [93]:
# Returns json text ready to be saved as file
def get_posicao( cod_linha ):
    res = s.get(url+'/Posicao?codigoLinha='+str(cod_linha))
    return res.text

In [62]:
cod_linhas = []
with open('../dados/gtfs-20170411/routes.txt', 'r') as f:
    routes = csv.reader(f)
    next(routes, None)  # ignore header
    for line in routes:
        cod_linhas.append(get_cod_linha(line[0]))

cod_linhas

[2486,
 34998,
 33599,
 2141,
 832,
 2495,
 33601,
 35070,
 2303,
 2306,
 2452,
 2453,
 33284,
 621,
 33427,
 34898,
 32897,
 1831,
 32898,
 32900,
 34580,
 2009,
 34778,
 32901,
 33287,
 841,
 661,
 35041,
 33610,
 33459,
 843,
 2304,
 35073,
 35219,
 531,
 534,
 33613,
 2476,
 35245,
 846,
 847,
 849,
 33619,
 34735,
 863,
 1937,
 666,
 864,
 34171,
 33624,
 33625,
 664,
 34569,
 34813,
 858,
 33627,
 33628,
 861,
 33630,
 33633,
 33439,
 672,
 33307,
 33310,
 543,
 33312,
 546,
 33441,
 33315,
 33316,
 552,
 33321,
 675,
 623,
 557,
 33393,
 34706,
 1782,
 33395,
 33396,
 634,
 635,
 637,
 638,
 1789,
 33398,
 33399,
 867,
 869,
 643,
 644,
 2046,
 34707,
 34584,
 649,
 33418,
 33421,
 654,
 558,
 560,
 33333,
 33636,
 34993,
 680,
 2450,
 2135,
 33479,
 892,
 893,
 895,
 33664,
 33665,
 1525,
 898,
 34872,
 2126,
 34904,
 2281,
 33667,
 33668,
 33670,
 33671,
 904,
 33673,
 35030,
 906,
 33132,
 2260,
 2261,
 2267,
 2269,
 2263,
 35038,
 2264,
 33135,
 2266,
 32972,
 259,
 260,
 13

In [96]:
while True:
    for linha in cod_linhas:
        with open('raw/'+str(linha)+'-'+str(time.time())+'.json', 'w') as f:
            f.write(get_posicao(linha))

ReadTimeout: HTTPConnectionPool(host='api.olhovivo.sptrans.com.br', port=80): Read timed out. (read timeout=None)